In [ ]:
%load_ext autoreload
%autoreload 2

## Libraries

In [ ]:
# Add root to path
import sys


sys.path.append("..")

from component.script.project import Project
from component.script.geo_utils import calculate_utm_rioxarray
from component.script.variables.models import RasterType

## Load project

In [ ]:
project_name = "nuevo3"

# Load the project from JSON
project = Project.load(project_name=project_name)


## Define base variable

The base variable will be uses as reference for all other variables in terms of resolution, crs and extent.

In [ ]:
# First, we list all the raster files in the project directory
project.raw_variables

In [ ]:
# We select the name of the variable we want to use

base_layer_name = "subj" # We can change this if our base has a different name
base_variable = project.raw_variables[base_layer_name]

In [ ]:
user_defined_epsg_code = None # or write "EPSG:3116"

In [ ]:
# If the user has not defined an EPSG code, we calculate it from the base variable
if user_defined_epsg_code is None:
    calculated_epsg = calculate_utm_rioxarray(
        base_variable.path
    )
    epsg_code = calculated_epsg
elif user_defined_epsg_code is not None:
    epsg_code = user_defined_epsg_code

print(f"Using EPSG:{epsg_code} for reprojection.")


In [ ]:
# Reproject the base variable to the calculated or user-defined EPSG code
# This will be the reference for all other variables
reprojected_base = base_variable.reproject(target_epsg=epsg_code)

In [ ]:
# We call a method to make this var the base of our project
reprojected_base.use_as_base_raster()

In [ ]:
project.base_raster

## Create the target variables

In [ ]:
calculate_forest_loss = True
calculate_forest_loss_stack = True

In [ ]:
from component.script.processing import get_forest_loss_calculated
from component.script.variables import LocalRasterVar

# Our goal is to train a model that fits the deforestation which ocurrend in a period of time
# For that, we need to create a target variable that represents the deforestation in that period

if calculate_forest_loss:

    forest_layers = project.filter_by_tags(["forest"], look_up_in="raw")
    forest_layers = [layer for layer in forest_layers.values()]
    
    forest_loss_layers = get_forest_loss_calculated(project, forest_layers)

    # now we add the new layers to the project
    for layer in forest_loss_layers:
        layer: LocalRasterVar
        # Raw because we have not projected it yet
        layer.add_as_raw()


In [ ]:
project.filter_by_tags(["forest"], look_up_in="raw")

In [ ]:
forest_loss_layers

In [ ]:
project.filter_by_attrs("raw", tags="deforestation")

In [ ]:
forest_loss_layers[0]

## Reproject and Rasterize Vector Data

In [ ]:
project.raw_variables

In [ ]:
# Finally, we reproject all the variables to match the base variable
project.processed_variables = {} # Make sure we clear raw variables to avoid showing them twice
project.reproject_and_match_all()

In [ ]:
# Rasterize will also also set the new projection and match the base raster
project.rasterize_all()

## Reproject Raster Data

## Calculate distance

In [ ]:
project.processed_vars

In [ ]:
roads_projected_edge = (
    project.processed_vars["roads"]
        .reproject(target_epsg=epsg_code)
        .apply_post_processing("edge")
        .add_as_processed()
)

In [ ]:
project.processed_vars["roads_edge"].show()

In [ ]:
list(project.processed_vars.keys())

In [ ]:
roads_projected_dist = (
    project.processed_vars["centros_poblados"]
        .apply_post_processing("dist")
        .add_as_processed()
)

In [ ]:
project.processed_vars["centros_poblados_dist"].show()

# Double check the dataset

In [ ]:
project.base_raster.path

In [ ]:
coca_var = LocalRasterVar(
    name="coca",
    path="/home/dguerrero/1_modules/deforisk-jupyter-nb-v2/data/Datos_pruebas_degradacion/Variables_raster/DE_COCA.img",
    project=project,
    raster_type=RasterType.categorical
).reproject_and_match(project.base_raster.get_base_geobox()).add_as_processed()